### **SKIN TONE DETECTION**

In [ ]:
!pip install mtcnn
!pip install tensorflow==2.11.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00


**PRINTING THE SKIN COLOR**

In [33]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from sklearn.cluster import KMeans

# Load the trained model
model = load_model('/content/model.h5')  # Replace with your model path

# List of class names
classes = ['Fair_Light', 'Medium_Tan', 'Dark_Deep']

# Mapping dictionary for descriptive skin tone labels
descriptive_labels = {
    'Fair_Light': 'Fair / Light',
    'Medium_Tan': 'Medium / Tan',
    'Dark_Deep': 'Dark / Deep'
}

# Load the MTCNN face detection model
mtcnn = MTCNN()


def kmeans_dominant_color(image, k=3):
    """Extract the dominant color using K-Means clustering."""
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_[np.argmax(np.bincount(kmeans.labels_))]
    return dominant_color.astype(int)

    """Extract skin regions and find the dominant color."""
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    lower = np.array([0, 133, 77], dtype=np.uint8)
    upper = np.array([255, 173, 127], dtype=np.uint8)
    skin_mask = cv2.inRange(ycrcb, lower, upper)
    skin_pixels = cv2.bitwise_and(image, image, mask=skin_mask)
    dominant_color = np.mean(skin_pixels[skin_mask > 0], axis=0)
    return dominant_color.astype(int)


def predict_skin_tone(image_path):
    try:
        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image could not be read. Check the file path.")

        # Detect faces
        faces = mtcnn.detect_faces(image)
        if len(faces) > 0:
            # Select the largest face
            largest_face = max(faces, key=lambda f: f['box'][2] * f['box'][3])
            x, y, w, h = largest_face['box']
            detected_face = image[y:y + h, x:x + w]

            # Resize the detected face to the desired input shape
            resized_face = cv2.resize(detected_face, (120, 90))

            # Preprocess the detected face for classification
            preprocessed_face = tf.keras.applications.mobilenet_v2.preprocess_input(resized_face[np.newaxis, ...])

            # Predict the class of the face
            predictions = model.predict(preprocessed_face)
            predicted_class_idx = np.argmax(predictions)
            predicted_class = classes[predicted_class_idx]

            # Get the descriptive label from the mapping dictionary
            descriptive_label = descriptive_labels[predicted_class]
            print(f'Predicted Skin Tone: {descriptive_label}')

            # Save the dominant colors using each method
            # 1. K-Means Clustering
            kmeans_color = kmeans_dominant_color(detected_face)
            kmeans_square = np.full((100, 100, 3), kmeans_color, dtype=np.uint8)
            kmeans_path = "/content/color_result/kmeans_dominant_color.jpg"
            cv2.imwrite(kmeans_path, kmeans_square)
            print(f"K-Means Dominant Color (BGR): {kmeans_color}")
            print(f"Saved to: {kmeans_path}")

        else:
            print('No face detected in the uploaded image.')
    except Exception as e:
        print(f'Error processing the image: {e}')


# Prompt the user for an image file path
if __name__ == "__main__":
    image_path = input("Enter the path to the image file: ")
    predict_skin_tone(image_path)


Enter the path to the image file: /content/test3.png
1/1 [==============================] - 1s 863ms/step
Predicted Skin Tone: Fair / Light
K-Means Dominant Color (BGR): [130 160 218]
Saved to: /content/color_result/kmeans_dominant_color.jpg


In [39]:
import numpy as np
import csv

# Load the dataset from a CSV file
def load_dataset(file_path):
    """
    Load the dataset from a CSV file with the specified structure.
    """
    dataset = []
    with open(file_path, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            skin_tone = [int(row["Skin_Tone_R"]), int(row["Skin_Tone_G"]), int(row["Skin_Tone_B"])]
            suited_colors = [
                [int(value) for value in color.split(",")]
                for color in row["Suited_Colors"].split(";")
            ]
            dataset.append({"skin_tone": skin_tone, "suited_colors": suited_colors})
    return dataset

# Load the dataset from the specified path
dataset = load_dataset("/content/expanded_skin_tone_colors.csv")

def find_closest_skin_tones(detected_rgb, threshold=30):
    """
    Find all skin tones within a threshold distance of the detected tone.
    """
    close_tones = []
    for entry in dataset:
        skin_tone = np.array(entry["skin_tone"])
        distance = np.linalg.norm(skin_tone - detected_rgb)  # Euclidean distance
        if distance <= threshold:
            close_tones.append({"skin_tone": entry["skin_tone"], "suited_colors": entry["suited_colors"], "distance": distance})
    # Sort by distance (closest first)
    close_tones = sorted(close_tones, key=lambda x: x["distance"])
    return close_tones

def recommend_colors_multiple(detected_rgb, threshold=30):
    """
    Recommend clothing colors based on detected skin tone and nearby tones.
    """
    close_tones = find_closest_skin_tones(detected_rgb, threshold)
    if close_tones:
        print("\nRecommended Clothing Colors (RGB):")
        all_recommendations = set()  # Use a set to avoid duplicate colors
        for tone in close_tones:
            print(f" ")
            for color in tone["suited_colors"]:
                all_recommendations.add(tuple(color))  # Add colors to the set
        for color in all_recommendations:
            print(list(color))  # Print as list for readability
    else:
        print("No matching skin tones found within the threshold.")

# Example: Detected skin tone
detected_rgb = np.array([215, 180, 130])  # Example detected tone
recommend_colors_multiple(detected_rgb, threshold=40)  # Adjust threshold as needed



Recommended Clothing Colors (RGB):
 
 
[186, 85, 211]
[255, 127, 80]
[32, 178, 170]
[70, 130, 180]
[240, 230, 140]
[255, 204, 0]
[255, 140, 0]
[245, 222, 179]
[46, 139, 87]
[255, 215, 0]


### **MINIMUM PRICE FINDER**

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter

# Load a pre-trained model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()

# Define the image transformation
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[:3, :, :]),  # Select the first 3 channels (RGB)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define a mapping from model output to item categories
category_mapping = {
    0: 'T-Shirts',
    1: 'Shirts',
    2: 'Jeans',
    3: 'Skirts',
    4: 'Dresses',
    5: 'Jackets',
    6: 'Shorts'
}

# Define a function to identify the dominant colors in the image
def identify_colors(image_path, num_colors=3):
    image = Image.open(image_path)
    image = image.resize((50, 50))  # Resize for faster processing

    # Convert the image to RGB format if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = np.array(image).reshape(-1, 3)

    # Use K-means clustering to find dominant colors
    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)

    # Define a comprehensive set of color ranges
    color_mapping = {
        'Red': [255, 0, 0],
        'Green': [0, 255, 0],
        'Blue': [0, 0, 255],
        'Black': [0, 0, 0],
        'White': [255, 255, 255],
        'Yellow': [255, 255, 0],
        'Purple': [128, 0, 128],
        'Orange': [255, 165, 0],
        'Pink': [255, 192, 203],
        'Brown': [165, 42, 42],
        'Gray': [128, 128, 128],
        'Light Blue': [173, 216, 230],
        'Dark Blue': [0, 0, 139],
        'Light Green': [144, 238, 144],
        'Dark Green': [0, 100, 0],
        'Light Red': [255, 182, 193],
        'Dark Red': [139, 0, 0],
        'Light Yellow': [255, 255, 224],
        'Dark Yellow': [204, 204, 0],
        'Cyan': [0, 255, 255],
        'Magenta': [255, 0, 255],
        'Maroon': [128, 0, 0],
        'Olive': [128, 128, 0],
        'Teal': [0, 128, 128],
        'Navy': [0, 0, 128],
        'Silver': [192, 192, 192],
        'Gold': [255, 215, 0],
        'Beige': [245, 245, 220],
        'Lavender': [230, 230, 250],
        'Turquoise': [64, 224, 208],
        'Peach': [255, 218, 185],
        'Mint': [189, 252, 201],
        'Coral': [255, 127, 80],
        'Indigo': [75, 0, 130],
        'Violet': [238, 130, 238],
        'Khaki': [240, 230, 140],
        'Plum': [221, 160, 221],
        'Crimson': [220, 20, 60],
        'Salmon': [250, 128, 114],
        'Chocolate': [210, 105, 30],
        'Tan': [210, 180, 140],
        'Orchid': [218, 112, 214],
        'Azure': [240, 255, 255],
        'Ivory': [255, 255, 240],
        'Lime': [0, 255, 0],
        'Periwinkle': [204, 204, 255],
        'Chartreuse': [127, 255, 0],
        'Aquamarine': [127, 255, 212],
        'Fuchsia': [255, 0, 255],
        'Wheat': [245, 222, 179],
        'Lemon': [255, 250, 205],
        'Sienna': [160, 82, 45],
        'Mauve': [224, 176, 255],
        'Rose': [255, 228, 225],
        'Burgundy': [128, 0, 32],
        'Mustard': [255, 219, 88],
        'Copper': [184, 115, 51],
        'Bronze': [205, 127, 50],
        'Ruby': [224, 17, 95],
        'Emerald': [80, 200, 120],
        'Sapphire': [15, 82, 186],
        'Amber': [255, 191, 0],
        'Jade': [0, 168, 107],
        'Pearl': [234, 224, 200],
        'Onyx': [53, 56, 57],
        'Obsidian': [78, 61, 43],
        'Topaz': [255, 200, 124],
        'Garnet': [115, 54, 53],
        'Amethyst': [153, 102, 204],
        'Citrine': [228, 208, 10],
        'Opal': [168, 195, 188],
        'Quartz': [217, 217, 243],
        'Spinel': [114, 47, 55],
        'Tourmaline': [48, 213, 200],
        'Zircon': [95, 158, 160]
    }

    # Find the closest colors using Euclidean distance
    def closest_color(color):
        return min(color_mapping.keys(), key=lambda c: np.linalg.norm(color - np.array(color_mapping[c])))

    identified_colors = [closest_color(color) for color in colors]

    # Count the most common colors
    color_counts = Counter(identified_colors)
    most_common_colors = [color for color, count in color_counts.most_common(num_colors)]

    return most_common_colors

def classify_image(image_path):
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        outputs = model(image)

    _, predicted = outputs.max(1)
    category = category_mapping.get(predicted.item(), 'T-Shirts')  # Default to 'T-Shirts' if not found

    # Identify the colors of the item
    colors = identify_colors(image_path)

    return category, colors

def extract_features(image_path):
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)

    with torch.no_grad():
        features = model(image)

    return features

# Load the fixed sample dataset
df = pd.read_csv('updated_sample_database_extended.csv')

# Find the minimum price for each item
df['Min Price'] = df[['Platform A', 'Platform B', 'Platform C']].min(axis=1)
df['Best Platform'] = df[['Platform A', 'Platform B', 'Platform C']].idxmin(axis=1)

# Function to find similar items and compare prices
def compare_similar_items(item_category, item_colors):
    similar_items = df[(df['Item'] == item_category) & (df['Color'].isin(item_colors))]

    # Sort by Min Price and select top 5 options with most similarity and best price options
    top_similar_items = similar_items.sort_values(by='Min Price').head(5)

    return top_similar_items

def main(image_path):
    # Step 1: Classify the image and identify the colors
    category, colors = classify_image(image_path)
    print(f'Predicted category: {category}, Colors: {colors}')

    # Step 2: Extract features (optional, for more advanced similarity search)
    features = extract_features(image_path)

    # Step 3: Compare prices of similar items and select top 5 options
    top_similar_items = compare_similar_items(category, colors)
    print(top_similar_items)

if __name__ == "__main__":
    image_path = 'test3.png'
    main(image_path)


### **MATCHING OUTFIT**